In [ ]:
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import torch
from cmcrameri import cm

## Load Geometry

In [ ]:
plt.style.use(Path("../meta/colorblind_friendly.mplstyle"))
matplotlib.rcParams.update({"font.family": "serif", "font.size": 8})


# station locations
stationlist_all = pl.read_csv("../meta/stations_all.csv")
stations_all = torch.tensor(
    np.vstack((stationlist_all["X"].to_numpy(), stationlist_all["Y"].to_numpy())).T
)

# receiver stations
stationlist_rcv = pl.read_csv("../meta/stations_receivers.csv")
names_receivers = stationlist_rcv["station"]
stations_receivers = torch.tensor(
    np.vstack((stationlist_rcv["X"].to_numpy(), stationlist_rcv["Y"].to_numpy())).T
)

master_idx = names_receivers.to_numpy().tolist().index("OMV.GDT")

_cm = 1 / 2.54

fig, ax = plt.subplots(figsize=(1.45 * 9 * _cm, 9 * _cm))
ax.scatter(*stations_all.T, s=5, lw=0, c="#CCC")
ax.scatter(*stations_receivers.T, s=5, lw=0)
ax.scatter(*stations_receivers[master_idx].T, s=100, marker="v", lw=0.5, ec="k")

ax.set(xlim=(-11, 11), ylim=(-8, 7), aspect="equal")

## Load Correlations

Please follow the instructions in `../data/` if the correlation files are missing.

In [ ]:
# load correlations
correlations_filt_data = torch.load(
    "../data/correlations_for_c1_filt_data.pt", weights_only=False
)
correlations_filt_synth = torch.load(
    "../data/correlations_for_c1_filt_synth_both.pt", weights_only=False
)

length_of_oneside = 300
sampling_rate = 5
lapse_times = torch.arange(
    -length_of_oneside, length_of_oneside + 1 / sampling_rate, 1 / sampling_rate
)

lapse_times.shape

## Manuscript Figure 3

In [ ]:
fig, axs = plt.subplots(
    2, 3, figsize=(18 * _cm, 2 / 3 * 18 * _cm), sharex=True, sharey=True
)

# extract 0-lag focal spot to determine normalisation
# in field data, autocorrelation is much stronger, remove for normalisation
data_focal_spot = correlations_filt_data[..., lapse_times.abs().argmin()].clone()
data_focal_spot[master_idx] = 0
synth_focal_spot = correlations_filt_synth[..., lapse_times.abs().argmin()].clone()
synth_focal_spot[master_idx] = 0

maxs_for_norm = [data_focal_spot.abs().max(), synth_focal_spot.abs().max()]

# plot data for different time windows
times_to_plot = [-1, 0, 1] * 2
labels = ("a)", "b)", "c)", "d)", "e)", "f)")
for ax, time_to_plot, label in zip(axs.flatten(), times_to_plot, labels):
    ax.set_title(f"{label}", loc="left", fontsize=10, pad=5)
    ax.set_xlim(-4.5, 4.5)
    ax.set_ylim(-4.5, 4.5)
    ax.set_aspect("equal")
    ax.set_yticks([-4, 0, 4])
    ax.set_xticks([-4, 0, 4])
    ax.set_xticklabels([-4, 0, 4], fontsize=10)
    ax.set_yticklabels([-4, 0, 4], fontsize=10)

    if ax in axs[0]:
        focal_spot = correlations_filt_data[
            ..., (lapse_times - time_to_plot).abs().argmin()
        ]
        focal_spot[master_idx] = 0
        prefix = "Observed"
        max_norm = maxs_for_norm[0]
        _cmap = cm.vik
    else:
        focal_spot = correlations_filt_synth[
            ..., (lapse_times - time_to_plot).abs().argmin()
        ]
        prefix = "Simulated"
        max_norm = maxs_for_norm[1]
        _cmap = cm.broc

    # normalise amplitudes to global max
    plot_fs = focal_spot / max_norm

    sct = ax.scatter(
        *stations_receivers.T,
        c=plot_fs,
        cmap=_cmap,
        s=10,
        vmin=-1,
        vmax=1,
        lw=0,
    )

    # in last column, add colorbar
    if time_to_plot == times_to_plot[-1]:
        x0, y0, w, h = ax.get_position().bounds
        cbar_ax = fig.add_axes([x0 + w + 0.01, y0, 0.01, h])
        cbar = fig.colorbar(sct, cax=cbar_ax, orientation="vertical")
        cbar.set_label(f"{prefix} " + r"$C_1$" + " amplitudes", fontsize=10)
        cbar.ax.tick_params(labelsize=10)
        # set ticks to -1, 0, 1
        cbar.set_ticks([-1, 0, 1])
        cbar.set_ticklabels(["-1", "0", "1"])

    # indicate lapse time
    t = ax.text(
        0.05,
        0.95,
        rf"$\tau$ = {time_to_plot} s",
        ha="left",
        va="top",
        transform=ax.transAxes,
        fontsize=10,
        bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.7),
    )

    # highlight master station
    ax.scatter([0], [0], marker="v", ec="k", s=100, lw=1, c="#FFA90E")

for ax in axs[1]:
    ax.set_xlabel("Distance [km]", labelpad=0, fontsize=10)
for ax in axs[:, 0]:
    ax.set_ylabel("Distance [km]", labelpad=0, fontsize=10)

fig.savefig("../figures/figure3.png", dpi=300, bbox_inches="tight")
